<a href="https://colab.research.google.com/github/HenriqueCCdA/bootCampAluraDataScience/blob/master/modulo4/extra/Explorando_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np

In [418]:
def colunas_selecionadas(colunas: str,
                         colunas_excluidas=[],
                         f_diff: bool=True)->str:

  colunas_excluidas+=["ICU", "WINDOW"]

  colunas_selecionadas = []
  for name in colunas:
    if ("DIFF" in name) and f_diff:
      continue
    elif name in colunas_excluidas:
      continue
    else:
      colunas_selecionadas.append(name)
   
  return colunas_selecionadas


def proporcao_y(y, name:str):
  p = y.value_counts(normalize=True)

  print(f"Proporcao do {name}")
  for l , v in zip(p.index, p.values):
    print(f"Campo {l} ->  {v:.2f}")

def lendo_dados(path):

  dados = pd.read_csv(path)

  nl, nc = dados.shape
  print(f"Número de linhas  :{nl}")
  print(f"Número de colunas :{nc}")

  return dados

def score(y_pre, y_test, verbose: bool = True)-> (float, float):
  tx = sum(y_pre == y_test)/len(y_test)

  if verbose:
    print("*************************")
    print(f"Taxa de acerto {tx:.2f}")
    print("*************************")

  return tx


def run_LogisticRegression(dados, 
                           seed, 
                           colunas_excluidas= [], 
                           f_diff: bool = True,
                           verbose: bool = True)-> float:
  
  x = dados[colunas_selecionadas(dados.columns, colunas_excluidas, f_diff)]
  y = dados.iloc[:,-1]
  
  np.random.seed(seed)

  x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y)

  if verbose:
    print(x.columns.tolist())
    print("********************************************")
    print(f"Numero de variaveis          = {x.shape[1]}")
    print(f"Numero de amostras de treino = {x.shape[0]}")
    print("********************************************")

  if verbose:
    proporcao_y(y      , "y")
    proporcao_y(y_test , "y_test")
    proporcao_y(y_train, "y_train")

  modelo = LogisticRegression(max_iter=5000, verbose=0, tol=1e-8, C=0.1)

  modelo.fit(x_train, y_train)

  y_hat = modelo.predict(x_train)
  y_pred = modelo.predict(x_test)

  s_train = score(y_hat , y_train, verbose)
  s_test   = score(y_pred, y_test, verbose)

  return s_test, s_train

In [410]:
dados = lendo_dados("dados_tratados_por_paciente.csv")

Número de linhas  :351
Número de colunas :231


In [411]:
dados.head()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,...,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,0.111111,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,0.898990,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
1,2,0,-1.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.978029,-1.0,-0.489712,-0.685470,-0.048218,-0.645951,0.357143,0.935673,-0.506173,-0.815385,-0.056604,-0.517241,0.357143,0.947368,-0.525773,-0.5125,-0.111111,-0.714286,0.604396,0.959596,-0.435897,-0.491892,0.000000,-0.575758,0.101449,1.000000,-0.547826,-0.533742,-0.603053,-0.764706,-1.000000,-0.959596,-0.515528,-0.351328,-0.747001,-0.756272,-1.000000,-0.961262,0-2,1
2,3,0,-0.333333,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.263158,-0.263158,-0.263158,-0.263158,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.972789,-0.972789,...,-0.978029,-1.0,0.012346,-0.369231,-0.528302,-0.457627,-0.285714,0.684211,0.012346,-0.369231,-0.528302,-0.448276,-0.285714,0.684211,0.175258,-0.1125,-0.384615,-0.357143,0.208791,0.878788,-0.299145,-0.556757,-0.626866,-0.515152,-0.420290,0.684211,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
3,4,0,-1.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.935113,-0.935113,...,-1.000000,-1.0,0.333333,-0.153846,0.160377,-0.593220,0.285714,0.868421,0.333333,-0.153846,0.160377,-0.586207,0.285714,0.868421,0.443299,0.0000,0.196581,-0.571429,0.538462,0.939394,-0.076923,-0.351351,-0.044776,-0.575758,0.072464,0.894737,-1.000000,-0.877301,-0.923664,-0.882353,-0.952381,-0.979798,-1.000000,-0.883669,-0.956805,-0.870968,-0.953536,-0

In [412]:
run_LogisticRegression(dados, seed=146731, f_diff=False, verbose = True)

['PATIENT_VISIT_IDENTIFIER', 'AGE_ABOVE65', 'AGE_PERCENTIL', 'GENDER', 'DISEASE GROUPING 1', 'DISEASE GROUPING 2', 'DISEASE GROUPING 3', 'DISEASE GROUPING 4', 'DISEASE GROUPING 5', 'DISEASE GROUPING 6', 'HTN', 'IMMUNOCOMPROMISED', 'OTHER', 'ALBUMIN_MEDIAN', 'ALBUMIN_MEAN', 'ALBUMIN_MIN', 'ALBUMIN_MAX', 'ALBUMIN_DIFF', 'BE_ARTERIAL_MEDIAN', 'BE_ARTERIAL_MEAN', 'BE_ARTERIAL_MIN', 'BE_ARTERIAL_MAX', 'BE_ARTERIAL_DIFF', 'BE_VENOUS_MEDIAN', 'BE_VENOUS_MEAN', 'BE_VENOUS_MIN', 'BE_VENOUS_MAX', 'BE_VENOUS_DIFF', 'BIC_ARTERIAL_MEDIAN', 'BIC_ARTERIAL_MEAN', 'BIC_ARTERIAL_MIN', 'BIC_ARTERIAL_MAX', 'BIC_ARTERIAL_DIFF', 'BIC_VENOUS_MEDIAN', 'BIC_VENOUS_MEAN', 'BIC_VENOUS_MIN', 'BIC_VENOUS_MAX', 'BIC_VENOUS_DIFF', 'BILLIRUBIN_MEDIAN', 'BILLIRUBIN_MEAN', 'BILLIRUBIN_MIN', 'BILLIRUBIN_MAX', 'BILLIRUBIN_DIFF', 'BLAST_MEDIAN', 'BLAST_MEAN', 'BLAST_MIN', 'BLAST_MAX', 'BLAST_DIFF', 'CALCIUM_MEDIAN', 'CALCIUM_MEAN', 'CALCIUM_MIN', 'CALCIUM_MAX', 'CALCIUM_DIFF', 'CREATININ_MEDIAN', 'CREATININ_MEAN', 'CREATI

(0.6590909090909091, 0.7908745247148289)

In [413]:
lista_sementes = [ 61367,  78668,  76592,   49310,  2825,
                 	 34136,  14747,  65643,   94788, 31840,
                   54546,	 71546,  95580,   32717,  1918,	
                   20892,	 18289,	  6295,	   8375,  1648,
                   65323,	 58650,	 70938,	  41914,	7960,
                   73762,	 76050,	 22706,	  39405, 70837]

In [419]:
res_treino = []
res_teste  = []

for semente in lista_sementes:

  test1, train1 = run_LogisticRegression(dados, seed=semente, f_diff=False, verbose = False)
  test2, train2 = run_LogisticRegression(dados, seed=semente,
                       colunas_excluidas=["PATIENT_VISIT_IDENTIFIER"], 
                       f_diff=True, verbose = False)
  test3, train3 = run_LogisticRegression(dados, seed=semente,
                       colunas_excluidas=["PATIENT_VISIT_IDENTIFIER", "AGE_ABOVE65"], 
                       f_diff=True, verbose = False)
  test4, train4 = run_LogisticRegression(dados, seed=semente,
                       colunas_excluidas=["PATIENT_VISIT_IDENTIFIER", "AGE_PERCENTIL"], 
                       f_diff=True, verbose = False)
  
  res_teste.append((test1, test2, test3, test4))
  res_treino.append((train1, train2, train3, train4))

res_treino = np.array(res_treino)
res_teste  = np.array(res_teste)

In [420]:
res = pd.DataFrame(data={'sementes':lista_sementes,
                   'l1_test'       :res_teste[:,0],
                   'l1_train'      :res_treino[:,0],
                   'l2_test'       :res_teste[:,1],
                   'l2_train'      :res_treino[:,1],
                   'l3_test'       :res_teste[:,2],  
                   'l3_train'      :res_treino[:,2],                   
                   'l4_test'       :res_teste[:,3],                 
                   'l4_train'      :res_treino[:,3]
                   })
res.head()

,sementes,l1_test,l1_train,l2_test,l2_train,l3_test,l3_train,l4_test,l4_train
0,61367,0.681818,0.794677,0.681818,0.764259,0.693182,0.764259,0.681818,0.764259
1,78668,0.681818,0.775665,0.693182,0.764259,0.693182,0.768061,0.715909,0.749049
2,76592,0.784091,0.737643,0.761364,0.722433,0.761364,0.722433,0.761364,0.756654
3,49310,0.693182,0.764259,0.704545,0.752852,0.693182,0.745247,0.738636,0.730038
4,2825,0.727273,0.760456,0.715909,0.752852,0.715909,0.752852,0.704545,0.752852


In [421]:
col = res.columns[1:]
medias = res[col].mean()
medianas = res[col].median()
desvio_padrao = res[col].std()

In [422]:
pd.DataFrame(data={'medias'       :medias,
                   'mediana'      :medianas,               
                   'desvio_padrao':desvio_padrao
                   }).T

,l1_test,l1_train,l2_test,l2_train,l3_test,l3_train,l4_test,l4_train
medias,0.711364,0.762231,0.709470,0.750951,0.710985,0.751838,0.712879,0.750570
mediana,0.710227,0.764259,0.704545,0.752852,0.698864,0.754753,0.704545,0.752852
desvio_padrao,0.037936,0.014902,0.039667,0.016820,0.043213,0.016314,0.044257,0.016212
